In [26]:
from vnstock import *
from datetime import date, datetime, timedelta
import numpy as np

def fm_date(date):
    return date.strftime("%Y-%m-%d")

# Get list of dates for the last 365 days
def get_last_days(num_days):
    current_date = datetime.now()
    days_ago = current_date - timedelta(days=num_days)
    date_list = [days_ago + timedelta(days=i) for i in range(num_days)]
    # Format the dates as "%Y-%m-%d" and store them in a new list
    formatted_dates = [fm_date(date) for date in date_list]

    return formatted_dates

def get_stock_price(symbol, num_days):
    dates_train = get_last_days(num_days)
    df_his = stock_historical_data(symbol, dates_train[0], dates_train[len(dates_train) - 1], '1D', 'stock', source='TCBS')
    return df_his


data = stock_historical_data(symbol='TCB', start_date='2023-06-01', end_date='2023-06-17', resolution='1D', type='stock', beautify=True, decor=True, source='TCBS')

In [27]:
import pandas as pd
import plotly.graph_objects as go


symbol = 'VPB'
num_days = 180
df = get_stock_price(symbol, num_days)
print(df.head())
df = df.reset_index()

def calculate_parabolic_sar(data, acceleration=0.02, max_acceleration=0.2):
    # Initialize the SAR values
    sar = data['high'][0]
    ep = data['low'][0]
    uptrend = True
    af = acceleration
    sar_values = [sar]

    # Calculate SAR for each data point
    for i in range(1, len(data)):
        try:
            high, low = data['high'][i], data['low'][i]

            # Flip the SAR if the high crosses above the SAR in downtrend or the low crosses below the SAR in uptrend
            if (uptrend and low <= sar) or (not uptrend and high >= sar):
                uptrend = not uptrend
                sar = ep
                af = acceleration

            # Update the Extreme Point (EP) and acceleration factor (AF)
            if (uptrend and high > ep) or (not uptrend and low < ep):
                ep = high if uptrend else low
                af = min(af + acceleration, max_acceleration)

            # Calculate the SAR for the next period
            sar = sar + af * (ep - sar)

            # Ensure the SAR does not cross the previous period's high/low
            if uptrend:
                sar = min(sar, data['low'][i-1], data['low'][i] if i < len(data) - 1 else sar)
            else:
                sar = max(sar, data['high'][i-1], data['high'][i] if i < len(data) - 1 else sar)

            sar_values.append(sar)
        except TypeError as e:
            print(e)
            sar_values.append(np.nan)

    return sar_values


# Calculate Parabolic SAR values
df['SAR'] = calculate_parabolic_sar(df)

# Create a Plotly figure
fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(x=df['time'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],))

# Add Parabolic SAR line
fig.add_trace(go.Scatter(x=df['time'], y=df['SAR'], mode='markers', marker=dict(color='blue'), name='Parabolic SAR'))

# Update layout
fig.update_layout(title='Parabolic SAR Chart',
                  xaxis_title='Date',
                  yaxis_title='Price')

# Show the plot
fig.show()


Time range is 179 days. Looping through 1 requests
          time   open   high    low  close   volume ticker
53  2023-05-24  18730  18780  18490  18490  8337114    VPB
54  2023-05-25  18440  18540  18300  18300  7300825    VPB
55  2023-05-26  18400  18400  18210  18250  6989347    VPB
56  2023-05-29  18300  18540  18300  18440  7045156    VPB
57  2023-05-30  18590  18590  18440  18540  8977201    VPB


In [28]:
print(df.index)

RangeIndex(start=0, stop=126, step=1)


In [29]:
def generate_signals(data):
    signals = pd.DataFrame(index=data.index)
    signals['signal'] = 0.0

    # Create a signal when the close price crosses the SAR
    signals['signal'][data['close'] > data['SAR']] = 1.0
    signals['signal'][data['close'] < data['SAR']] = -1.0

    return signals

signals = generate_signals(df)
print(signals)

     signal
0      -1.0
1      -1.0
2      -1.0
3      -1.0
4      -1.0
..      ...
121     1.0
122     1.0
123     1.0
124     1.0
125    -1.0

[126 rows x 1 columns]


In [30]:
import plotly.graph_objects as go

# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

# Add SAR line
fig.add_trace(go.Scatter(x=df.index, y=df['SAR'], mode='lines', name='SAR'))

# Add shapes for buy and sell signals
buy_signals = signals[signals['signal'] == 1.0]

print(buy_signals.index)
sell_signals = signals[signals['signal'] == -1.0]
fig.add_trace(go.Scatter(x=buy_signals.index, y=df['low'][buy_signals.index], mode='markers', name='Buy Signal', marker=dict(color='green', size=10, symbol='triangle-up')))
fig.add_trace(go.Scatter(x=sell_signals.index, y=df['high'][sell_signals.index], mode='markers', name='Sell Signal', marker=dict(color='red', size=10, symbol='triangle-down')))

# Set the title
fig.update_layout(title='Stock Price with Parabolic SAR and Trading Signals')

# Show the plot
fig.show()

Int64Index([  7,   8,   9,  10,  11,  17,  22,  23,  24,  25,  26,  27,  28,
             29,  30,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
             49,  50,  51,  52,  53,  54,  61,  72,  73,  74,  75,  76,  77,
             78,  79,  80,  81,  90,  91,  92,  93,  94,  95,  96,  97,  98,
             99, 100, 101, 102, 103, 118, 119, 120, 121, 122, 123, 124],
           dtype='int64')


In [31]:
# Initialize variables
position = None
buy_price = None
sell_price = None
profits = []

# Loop through each data point
for i in range(1, len(df)):
    # Check if we should buy
    if df['close'][i] > df['SAR'][i] and df['SAR'][i-1] > df['close'][i-1] and position is None:
        position = 'long'
        buy_price = df['close'][i]
        print(f'Bought at {buy_price}')

    # Check if we should sell
    elif df['close'][i] < df['SAR'][i] and df['SAR'][i-1] < df['close'][i-1] and position == 'long':
        position = None
        sell_price = df['close'][i]
        profit = sell_price - buy_price
        profits.append(profit)
        print(f'Sold at {sell_price}, Profit: {profit}')

# Print total profits
print(f'Total Profits: {sum(profits)}')


Bought at 18920
Sold at 18590, Profit: -330
Bought at 18640
Sold at 18540, Profit: -100
Bought at 19260
Sold at 18640, Profit: -620
Bought at 19490
Sold at 21020, Profit: 1530
Bought at 21110
Sold at 19640, Profit: -1470
Bought at 20020
Sold at 20880, Profit: 860
Bought at 20680
Sold at 20450, Profit: -230
Bought at 20350
Sold at 19300, Profit: -1050
Total Profits: -1410


In [32]:
import pandas as pd
import plotly.graph_objects as go

def calculate_momentum(data, period=14):
    data['Momentum'] = data['close'].rolling(window=period).mean()
    return data


# Calculate Momentum values
data = calculate_momentum(df)

# Create a Plotly figure
fig = go.Figure()

# Add line chart for closing prices
fig.add_trace(go.Scatter(x=data['time'], y=data['close'], mode='lines', name='Close'))

# Add Momentum line
fig.add_trace(go.Scatter(x=data['time'], y=data['Momentum'], mode='lines', name='Momentum'))

# Update layout
fig.update_layout(title='Momentum Indicator Chart',
                  xaxis_title='Date',
                  yaxis_title='Value')

# Show the plot
fig.show()


In [33]:
company_overview('TCB').T

,0
ticker,TCB
exchange,HOSE
industry,Ngân hàng
companyType,NH
noShareholders,1901
foreignPercent,0.225
outstandingShare,3517.2
issueShare,3517.2
establishedYear,1993
noEmployees,11742


In [34]:
company_news(symbol='TCB', page_size=10, page=0)

,ticker,price,priceChange,priceChangeRatio,priceChangeRatio1W,priceChangeRatio1M,id,title,source,publishDate
0,TCB,31700,150,0.005,0.006,-0.003,10987692,TCB: Thông báo ngày ĐKCC để thực hiện quyền đ...,HOSE,2023-11-16 09:51:00
1,TCB,30900,-150,-0.005,-0.003,-0.040,10985063,TCB: Thông báo thay đổi nhân sự công ty,HOSE,2023-11-13 16:14:00
2,TCB,30900,-150,-0.005,-0.003,-0.040,10984675,TCB: Thông báo và Nghị quyết HĐQT về ngày ĐKC...,HOSE,2023-11-13 11:09:00
3,TCB,31000,1650,0.056,0.049,-0.046,10977208,TCB: Thông báo phát hành cổ phiếu theo chương...,HOSE,2023-11-03 16:56:00
4,TCB,31000,1650,0.056,0.049,-0.046,10977209,TCB: Thông báo giao dịch cổ phiếu ESOP của Ng...,HOSE,2023-11-03 16:55:00
5,TCB,27700,-400,-0.014,-0.116,-0.164,10972963,TCB: CBTT nhận được công văn của UBCK về thay...,HOSE,2023-10-31 17:17:00
6,TCB,29550,-250,-0.008,-0.050,-0.094,10969212,TCB: Quyết định của HĐQT về việc thay đổi địa...,HOSE,2023-10-27 13:16:00
7,TCB,29550,-250,-0.008,-0.050,-0.094,10969214,TCB: Quyết định của HĐQT về việc thay đổi địa...,HOSE,2023-10-27 13:13:00
8,TCB,29800,-1400,-0.045,-0.028,-0.080,10968259,TCB: CBTT Nghị quyết HĐQT phê duyệt ký kết hợ...,HOSE,2023-10-26 16:17:00
9,TCB,29800,-1400,-0.045,-0.028,-0.080,10968260,TCB: CBTT Nghị quyết HĐQT phê duyệt hợp đồng ...,HOSE,2023-10-26 16:14:00
